In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import scvelo as scv
import scvi
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output

import pandas as pd

import os
import sys
import time
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
######Mouse mm10######
start_time=time.strftime("%Y_%m_%d-%I_%M_%S_%p")
print('start time:',start_time)

sample_names = ['ME8','ME9','ME10','ME11','ME12'] #specify the sample names
species = 'mouse' #specify the species
genome = 'mm10' #specify the genome
output_prefix = 'cellranger_related/cellranger_output/sc_rnaseq/' #specify the location of the cellranger output

cellranger_locs = []
velocyto_locs = []

for sample in sample_names: #simple loop to create a list with the location of the required files
    cellranger_locs = cellranger_locs + [output_prefix+species+'/'+genome+'/'+sample+'/outs/filtered_feature_bc_matrix/']
    velocyto_locs = velocyto_locs + [output_prefix+species+'/'+genome+'/'+sample+'/velocyto/'+sample+'.loom']

adata_files={}
adata_list=[]
print(time.strftime("%Y_%m_%d-%I_%M_%S_%p"),'Reading file(s)')
for i,j,k in zip(sample_names, cellranger_locs, velocyto_locs):
    adata_cellr = sc.read_10x_mtx(j)
    adata_cellr.obs.index = adata_cellr.obs.index.str[:-2]
    adata_cellr.var['gene_name'] = adata_cellr.var.index
    adata_cellr.var.index = adata_cellr.var['gene_ids']
    adata_veloc = sc.read(k)
    adata_veloc.obs.index = adata_veloc.obs.index.str.split(':').str[1]
    adata_veloc.obs.index = adata_veloc.obs.index.str[:-1]
    adata_veloc.var['gene_name'] = adata_veloc.var.index
    adata_veloc.var.index = adata_veloc.var['Accession']
    adata_merge = scv.utils.merge(adata_cellr, adata_veloc)
    adata_files['adata'+i]=adata_merge
    adata_files['adata'+i].obs['sample']=i
    adata_files['adata'+i].obs['barcode']=adata_files['adata'+i].obs.index
    adata_files['adata'+i].obs.index=adata_files['adata'+i].obs['barcode']+'.'+adata_files['adata'+i].obs['sample']
    adata_list.append('adata'+i)
    del adata_cellr
    del adata_veloc
    del adata_merge
    gc.collect()
print(time.strftime("%Y_%m_%d-%I_%M_%S_%p"),'Finished Reading file(s)')